<a href="https://colab.research.google.com/github/markaaronslater/recurrent-NMT/blob/master/pos_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stanza

     |████████████████████████████████| 235kB 4.5MB/s 


In [2]:
%load_ext autoreload
%autoreload 2

In [46]:
from pos_tags import *
from preprocess import *


In [5]:
import stanza

stanza.download('de')
stanza.download('en')

src_processor = stanza.Pipeline(lang='de', processors='tokenize,mwt,pos')
trg_processor = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')


2020-09-07 22:56:03 INFO: Downloading default packages for language: de (German)...
2020-09-07 22:57:11 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-09-07 22:57:11 INFO: Downloading default packages for language: en (English)...
2020-09-07 22:57:45 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-09-07 22:57:45 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |

2020-09-07 22:57:45 INFO: Use device: gpu
2020-09-07 22:57:45 INFO: Loading: tokenize
2020-09-07 22:57:55 INFO: Loading: mwt
2020-09-07 22:57:55 INFO: Loading: pos
2020-09-07 22:57:56 INFO: Done loading processors!
2020-09-07 22:57:56 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-09-07 22:57:56 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
path = '/content/gdrive/My Drive/iwslt16_en_de/'

In [29]:
#load_docs2(path, all_corpuses)
#load_docs2(path, all_corpuses, num=5)
corpuses = load_docs3(path, "train.de", "train.en", "dev.de", "dev.en", "test.de", num=5)

In [30]:
#print_corpuses2(all_corpuses, num=10)
print_corpuses(corpuses, num=5)

David Gallo: Das ist Bill Lange. Ich bin Dave Gallo.
Wir werden Ihnen einige Geschichten über das Meer in Videoform erzählen.
Wir haben ein paar der unglaublichsten Aufnahmen der Titanic, die man je gesehen hat,, und wir werden Ihnen nichts davon zeigen.
Die Wahrheit ist, dass die Titanic – obwohl sie alle Kinokassenrekorde bricht – nicht gerade die aufregendste Geschichte vom Meer ist.
Ich denke, das Problem ist, dass wir das Meer für zu selbstverständlich halten.

David Gallo: This is Bill Lange. I'm Dave Gallo.
And we're going to tell you some stories from the sea here in video.
We've got some of the most incredible video of Titanic that's ever been seen, and we're not going to show you any of it.
The truth of the matter is that the Titanic -- even though it's breaking all sorts of box office records -- it's not the most exciting story from the sea.
And the problem, I think, is that we take the ocean for granted.

Als ich in meinen 20ern war, hatte ich meine erste Psychotherapie-Pat

In [23]:
lengths = corpus_lengths(corpuses)
print(lengths)

{'train.de': 12, 'train.en': 12, 'dev.de': 12, 'dev.en': 12, 'test.de': 12}


In [67]:
lookup_tables = create_lookup_tables(all_corpuses)

In [43]:
trg_train_to_line = lookup_tables["sent_to_line"]["train.en"]
i = 0
for sent in trg_train_to_line:
  if i == 5:
    break
  print(sent, trg_train_to_line[sent])
  i += 1




David Gallo: This is Bill Lange. I'm Dave Gallo. 1
And we're going to tell you some stories from the sea here in video. 2
We've got some of the most incredible video of Titanic that's ever been seen, and we're not going to show you any of it. 3
The truth of the matter is that the Titanic -- even though it's breaking all sorts of box office records -- it's not the most exciting story from the sea. 4
And the problem, I think, is that we take the ocean for granted. 5


In [68]:
for line in get_sentences(lookup_tables, "dev.de", line_nums=[5,6,7]):
  print(line)

Und als ich das hörte, war ich erleichtert.
Meine Kommilitonin bekam nämlich einen Brandstifter als ersten Patienten.
Und ich bekam eine Frau in den 20ern, die über Jungs reden wollte.


In [69]:
sentences = ['Now when I heard this, I was so relieved.', 'I was a Ph.D. student in clinical psychology at Berkeley.', 'My classmate got an arsonist for her first client.']
for num in get_line_nums(lookup_tables, "dev.en", sentences):
  print(num)

5
2
6


In [70]:
sentences = ['Now when I heard this, I was so relieved.', 'I was a Ph.D. student in clinical psychology at Berkeley.', 'My classmate got an arsonist for her first client.']
for partner in get_partners(lookup_tables, "dev.en", sentences):
  print(partner)

Und als ich das hörte, war ich erleichtert.
Ich war Doktorandin und studierte Klinische Psychologie in Berkeley.
Meine Kommilitonin bekam nämlich einen Brandstifter als ersten Patienten.


In [71]:
sentences = ['Es ist mir egal, ob es Mountainbike ist, oder ein Buch zu lesen, oder ein Spiel zu Spielen,', 'entdecken Sie es neu. Weil Sie die Führungspersönlichkeiten sind, die Innovationsanführer sind, die Master der Ideen sind.']
for partner in get_partners(lookup_tables, "train.de", sentences):
  print(partner)


I don't care if it's mountain biking or reading a book or playing a game.
Rediscover that because you're the leaders, the innovation leaders, the thought leaders.


In [36]:

#process_corpuses(all_corpuses, src_processor, trg_processor)
#process_corpuses(all_corpuses, src_processor, trg_processor, print_every=100)

#process_corpuses(all_corpuses, src_processor, trg_processor, num=5)
process_corpuses(corpuses, src_processor, trg_processor, piece_size=2)

pieces: {'train.de': 3, 'train.en': 3, 'dev.de': 3, 'dev.en': 3, 'test.de': 3}

processing piece 1 of train.de...
saved to /content/gdrive/My Drive/iwslt16_en_de/processed_train.de_1

processing piece 2 of train.de...
saved to /content/gdrive/My Drive/iwslt16_en_de/processed_train.de_2

processing piece 3 of train.de...
saved to /content/gdrive/My Drive/iwslt16_en_de/processed_train.de_3

processing piece 1 of train.en...
saved to /content/gdrive/My Drive/iwslt16_en_de/processed_train.en_1

processing piece 2 of train.en...
saved to /content/gdrive/My Drive/iwslt16_en_de/processed_train.en_2

processing piece 3 of train.en...
saved to /content/gdrive/My Drive/iwslt16_en_de/processed_train.en_3

processing piece 1 of dev.de...
saved to /content/gdrive/My Drive/iwslt16_en_de/processed_dev.de_1

processing piece 2 of dev.de...
saved to /content/gdrive/My Drive/iwslt16_en_de/processed_dev.de_2

processing piece 3 of dev.de...
saved to /content/gdrive/My Drive/iwslt16_en_de/processed_dev.de

In [37]:
processed_corpuses = get_processed_corpuses("train.de", "train.en", "dev.de", "dev.en", "test.de")

pieces: {'train.de': 3, 'train.en': 3, 'dev.de': 3, 'dev.en': 3, 'test.de': 3}



In [ ]:
print(processed_corpuses)

In [38]:
print_processed_corpuses(processed_corpuses, num=12)

word: David		upos: PROPN	xpos: NE
word: Gallo		upos: PROPN	xpos: NE
word: :		upos: PUNCT	xpos: $.
word: Das		upos: PRON	xpos: PDS
word: ist		upos: AUX	xpos: VAFIN
word: Bill		upos: PROPN	xpos: NE
word: Lange		upos: PROPN	xpos: NE
word: .		upos: PUNCT	xpos: $.
########################################
word: Ich		upos: PRON	xpos: PPER
word: bin		upos: AUX	xpos: VAFIN
word: Dave		upos: PROPN	xpos: NE
word: Gallo		upos: PROPN	xpos: NE
word: .		upos: PUNCT	xpos: $.
########################################



word: Wir		upos: PRON	xpos: PPER
word: werden		upos: AUX	xpos: VAFIN
word: Ihnen		upos: PRON	xpos: PPER
word: einige		upos: PRON	xpos: PIAT
word: Geschichten		upos: NOUN	xpos: NN
word: über		upos: ADP	xpos: APPR
word: das		upos: DET	xpos: ART
word: Meer		upos: NOUN	xpos: NN
word: in		upos: ADP	xpos: APPR
word: Videoform		upos: NOUN	xpos: NN
word: erzählen		upos: VERB	xpos: VVINF
word: .		upos: PUNCT	xpos: $.
########################################



word: Wir		upos: PRON	xpos: PPER
wor

In [40]:
decased_corpuses = decase_corpuses(processed_corpuses)

In [42]:
print_corpuses(decased_corpuses)

['David', 'Gallo', ':', 'das', 'ist', 'Bill', 'Lange', '.', 'ich', 'bin', 'Dave', 'Gallo', '.']
['wir', 'werden', 'Ihnen', 'einige', 'Geschichten', 'über', 'das', 'Meer', 'in', 'Videoform', 'erzählen', '.']
['wir', 'haben', 'ein', 'paar', 'der', 'unglaublichsten', 'Aufnahmen', 'der', 'Titanic', ',', 'die', 'man', 'je', 'gesehen', 'hat', ',', ',', 'und', 'wir', 'werden', 'Ihnen', 'nichts', 'davon', 'zeigen', '.']
['die', 'Wahrheit', 'ist', ',', 'dass', 'die', 'Titanic', '–', 'obwohl', 'sie', 'alle', 'Kinokassenrekorde', 'bricht', '–', 'nicht', 'gerade', 'die', 'aufregendste', 'Geschichte', 'von', 'dem', 'Meer', 'ist', '.']
['ich', 'denke', ',', 'das', 'Problem', 'ist', ',', 'dass', 'wir', 'das', 'Meer', 'für', 'zu', 'selbstverständlich', 'halten', '.']

['David', 'Gallo', ':', 'this', 'is', 'Bill', 'Lange', '.', 'I', "'m", 'Dave', 'Gallo', '.']
['and', 'we', "'re", 'going', 'to', 'tell', 'you', 'some', 'stories', 'from', 'the', 'sea', 'here', 'in', 'video', '.']
['we', "'ve", 'got', 'so

In [47]:
ref_corpuses = get_references(decased_corpuses)
print_corpuses(ref_corpuses)

[['David', 'Gallo', ':', 'this', 'is', 'Bill', 'Lange', '.', 'I', "'m", 'Dave', 'Gallo', '.']]
[['and', 'we', "'re", 'going', 'to', 'tell', 'you', 'some', 'stories', 'from', 'the', 'sea', 'here', 'in', 'video', '.']]
[['we', "'ve", 'got', 'some', 'of', 'the', 'most', 'incredible', 'video', 'of', 'Titanic', 'that', "'s", 'ever', 'been', 'seen', ',', 'and', 'we', "'re", 'not', 'going', 'to', 'show', 'you', 'any', 'of', 'it', '.']]
[['the', 'truth', 'of', 'the', 'matter', 'is', 'that', 'the', 'Titanic', '--', 'even', 'though', 'it', "'s", 'breaking', 'all', 'sorts', 'of', 'box', 'office', 'records', '--', 'it', "'s", 'not', 'the', 'most', 'exciting', 'story', 'from', 'the', 'sea', '.']]
[['and', 'the', 'problem', ',', 'I', 'think', ',', 'is', 'that', 'we', 'take', 'the', 'ocean', 'for', 'granted', '.']]

[['when', 'I', 'was', 'in', 'my', '20s', ',', 'I', 'saw', 'my', 'very', 'first', 'psychotherapy', 'client', '.']]
[['I', 'was', 'a', 'Ph.D.', 'student', 'in', 'clinical', 'psychology', 'a

In [112]:
german_decase([src_processor('Sie haben etwas namens RNA.')], num=1)

[
  {
    "id": 1,
    "text": "Sie",
    "upos": "PRON",
    "xpos": "PPER",
    "feats": "Case=Nom|Number=Plur|Person=3|PronType=Prs",
    "misc": "start_char=0|end_char=3"
  },
  {
    "id": 2,
    "text": "haben",
    "upos": "VERB",
    "xpos": "VAFIN",
    "feats": "Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin",
    "misc": "start_char=4|end_char=9"
  },
  {
    "id": 3,
    "text": "etwas",
    "upos": "ADV",
    "xpos": "ADV",
    "misc": "start_char=10|end_char=15"
  },
  {
    "id": 4,
    "text": "namens",
    "upos": "ADP",
    "xpos": "APPR",
    "misc": "start_char=16|end_char=22"
  },
  {
    "id": 5,
    "text": "RNA",
    "upos": "PROPN",
    "xpos": "NE",
    "feats": "Case=Acc|Gender=Masc|Number=Sing",
    "misc": "start_char=23|end_char=26"
  },
  {
    "id": 6,
    "text": ".",
    "upos": "PUNCT",
    "xpos": "$.",
    "misc": "start_char=26|end_char=27"
  }
]


In [99]:
#print_processed_corpus(all_corpuses["src"]["train.de"], num=1)

TypeError: ignored

In [ ]:
# with open(path + 'train.en', mode='rt', encoding='utf-8') as f:
#   train_en = f.read().strip().split('\n')

# with open(path + 'train.de', mode='rt', encoding='utf-8') as f:
#   train_de = f.read().strip().split('\n')

In [ ]:
# num = 5
# for sent in train_en[:num]:
#   print(sent)
# print()
# num = 5
# for sent in train_de[:num]:
#   print(sent)


David Gallo: This is Bill Lange. I'm Dave Gallo.
And we're going to tell you some stories from the sea here in video.
We've got some of the most incredible video of Titanic that's ever been seen, and we're not going to show you any of it.
The truth of the matter is that the Titanic -- even though it's breaking all sorts of box office records -- it's not the most exciting story from the sea.
And the problem, I think, is that we take the ocean for granted.

David Gallo: Das ist Bill Lange. Ich bin Dave Gallo.
Wir werden Ihnen einige Geschichten über das Meer in Videoform erzählen.
Wir haben ein paar der unglaublichsten Aufnahmen der Titanic, die man je gesehen hat,, und wir werden Ihnen nichts davon zeigen.
Die Wahrheit ist, dass die Titanic – obwohl sie alle Kinokassenrekorde bricht – nicht gerade die aufregendste Geschichte vom Meer ist.
Ich denke, das Problem ist, dass wir das Meer für zu selbstverständlich halten.


In [ ]:
# for sentence in train_en[:5]:
#   doc = nlp_en(sentence)
#   for sent in doc.sentences:
#     for word in sent.words: 
#       print(f'word: {word.text}\t\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}')
#     print()
#   print()
#   print()
#   print()


word: David		upos: PROPN	xpos: NNP	feats: Number=Sing
word: Gallo		upos: PROPN	xpos: NNP	feats: Number=Sing
word: :		upos: PUNCT	xpos: :	feats: _

word: This		upos: PRON	xpos: DT	feats: Number=Sing|PronType=Dem
word: is		upos: AUX	xpos: VBZ	feats: Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
word: Bill		upos: PROPN	xpos: NNP	feats: Number=Sing
word: Lange		upos: PROPN	xpos: NNP	feats: Number=Sing
word: .		upos: PUNCT	xpos: .	feats: _

word: I		upos: PRON	xpos: PRP	feats: Case=Nom|Number=Sing|Person=1|PronType=Prs
word: 'm		upos: AUX	xpos: VBP	feats: Mood=Ind|Tense=Pres|VerbForm=Fin
word: Dave		upos: PROPN	xpos: NNP	feats: Number=Sing
word: Gallo		upos: PROPN	xpos: NNP	feats: Number=Sing
word: .		upos: PUNCT	xpos: .	feats: _




word: And		upos: CCONJ	xpos: CC	feats: _
word: we		upos: PRON	xpos: PRP	feats: Case=Nom|Number=Plur|Person=1|PronType=Prs
word: 're		upos: AUX	xpos: VBP	feats: Mood=Ind|Tense=Pres|VerbForm=Fin
word: going		upos: VERB	xpos: VBG	feats: Tense=Pres|VerbForm

In [12]:
s = "It's called the United States Memory Championship."
doc = trg_processor(s)
for sent in doc.sentences:
  for word in sent.words: 
    print(f'word: {word.text}\t\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}')
  print()

word: It		upos: PRON	xpos: PRP	feats: Case=Nom|Gender=Neut|Number=Sing|Person=3|PronType=Prs
word: 's		upos: AUX	xpos: VBZ	feats: Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
word: called		upos: VERB	xpos: VBN	feats: Tense=Past|VerbForm=Part|Voice=Pass
word: the		upos: DET	xpos: DT	feats: Definite=Def|PronType=Art
word: United		upos: PROPN	xpos: NNP	feats: Number=Sing
word: States		upos: PROPN	xpos: NNP	feats: Number=Sing
word: Memory		upos: PROPN	xpos: NNP	feats: Number=Sing
word: Championship		upos: PROPN	xpos: NNP	feats: Number=Sing
word: .		upos: PUNCT	xpos: .	feats: _



In [89]:
s = 'Sie übernimmt Kontrolle über diese Zellen und verwandelt sie in Virenfabriken, die platzen alle, eine riesige Bakteriengroßfamilie, die nun alle sterben, weil Viren aus ihren Eingeweiden strömen, und die Viren übernehmen das Bakterium.'
doc = src_processor(s)
for sent in doc.sentences:
  for word in sent.words: 
    print(f'word: {word.text}\t\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}')
  print()

word: Sie		upos: PRON	xpos: PPER	feats: Case=Nom|Gender=Fem|Number=Sing|Person=3|PronType=Prs
word: übernimmt		upos: VERB	xpos: VVFIN	feats: Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
word: Kontrolle		upos: NOUN	xpos: NN	feats: Case=Acc|Gender=Fem|Number=Sing
word: über		upos: ADP	xpos: APPR	feats: _
word: diese		upos: PRON	xpos: PDAT	feats: Case=Acc|Gender=Fem|Number=Plur|PronType=Dem
word: Zellen		upos: NOUN	xpos: NN	feats: Case=Acc|Gender=Fem|Number=Plur
word: und		upos: CCONJ	xpos: KON	feats: _
word: verwandelt		upos: VERB	xpos: VVFIN	feats: Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
word: sie		upos: PRON	xpos: PPER	feats: Case=Acc|Number=Plur|Person=3|PronType=Prs
word: in		upos: ADP	xpos: APPR	feats: _
word: Virenfabriken		upos: NOUN	xpos: NN	feats: Case=Dat|Gender=Fem|Number=Plur
word: ,		upos: PUNCT	xpos: $,	feats: _
word: die		upos: PRON	xpos: PRELS	feats: Case=Nom|Number=Plur|PronType=Rel
word: platzen		upos: VERB	xpos: VVFIN	feats: Mood=Ind|Number=Plur|

In [ ]:
# for sentence in train_de[:5]:
#   doc = nlp_de(sentence)
#   for sent in doc.sentences:
#     for word in sent.words: 
#       print(f'word: {word.text}\t\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}')
#     print()
#   print()
#   print()
#   print()

word: David		upos: PROPN	xpos: NE	feats: Case=Nom|Gender=Masc|Number=Sing
word: Gallo		upos: PROPN	xpos: NE	feats: Case=Nom|Gender=Masc|Number=Sing
word: :		upos: PUNCT	xpos: $.	feats: _
word: Das		upos: PRON	xpos: PDS	feats: Case=Nom|Gender=Neut|Number=Sing|PronType=Dem
word: ist		upos: AUX	xpos: VAFIN	feats: Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
word: Bill		upos: PROPN	xpos: NE	feats: Case=Nom|Gender=Masc|Number=Sing
word: Lange		upos: PROPN	xpos: NE	feats: Case=Nom|Gender=Masc|Number=Sing
word: .		upos: PUNCT	xpos: $.	feats: _

word: Ich		upos: PRON	xpos: PPER	feats: Case=Nom|Number=Sing|Person=1|PronType=Prs
word: bin		upos: AUX	xpos: VAFIN	feats: Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin
word: Dave		upos: PROPN	xpos: NE	feats: Case=Nom|Gender=Masc|Number=Sing
word: Gallo		upos: PROPN	xpos: NE	feats: Case=Acc|Gender=Masc|Number=Sing
word: .		upos: PUNCT	xpos: $.	feats: _




word: Wir		upos: PRON	xpos: PPER	feats: Case=Nom|Number=Plur|Person=1|PronType=P